# Imports

In [1]:
import os; os.chdir('..')
import unidecode
import numpy as np
import pandas as pd
from pathlib import Path

In [4]:
PATH_OUT = Path('data/processed/criminality')

# Preprocess

In [7]:
PATH_OUT.mkdir(parents=True, exist_ok=True)

In [52]:
df_pop = pd.read_csv('data/processed/inei_series.csv')
df_pop = df_pop[df_pop.indicador.isin(['poblacion_total_urbana', 'poblacion_total_rural'])]
cols = [k for k,v in df_pop.iloc[:, 3:].sum().iteritems() if v > 0]
df_pop = df_pop.loc[:, ['ambito', 'indicador']+cols]
df_pop.to_csv(PATH_OUT/'population_inei.csv', index=False)
df_pop.head(2)

,ambito,indicador,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
4848,Amazonas,poblacion_total_urbana,99907.0,105146.0,110516.0,116024.0,121683.0,127506.0,133503.0,139654.0,145924.0,152277.0,158676.0,165116.0,171599.0,178105.0,184607.0,191075.0
4851,Amazonas,poblacion_total_rural,285334.0,283574.0,281422.0,278935.0,276165.0,273162.0,269925.0,266433.0,262705.0,258766.0,254638.0,250350.0,245909.0,241299.0,236515.0,231554.0


In [53]:
df = (pd.read_csv('data/raw/criminality/tipo-ubicacion.csv').
          drop('Periodo', axis=1).
          T.reset_index())
df.iat[0,0] = 'year'
df.iat[0,1] = 'departamento'
df.columns = df.iloc[0].apply(lambda x: unidecode.unidecode(x.lower().replace(' ', '-')))
df.drop(0, inplace=True)
df.reset_index(drop=True, inplace=True)
y = df.loc[0, 'year']
for i,row in df.iterrows():
    if row.year.isdigit():
        y = row.year
    row.year = y

df.year = df.year.astype(int)
df.departamento = (df.departamento.
                     str.title().
                     str.replace('Lima Provincias 2/', 'Lima Provincias').
                     str.replace('Provincia De Lima 1/', 'Lima').
                     str.replace('Provincia Constitucional Del Callao', 'Callao').
                     str.replace(' De ', ' de '))
df = df.melt(id_vars=['year', 'departamento'], var_name='variable')
df.value = df.value.str.replace(' ', '').fillna(0).astype(int)
df.to_csv(PATH_OUT/'tipo-ubicacion.csv', index=False)
df = df.set_index(['year', 'departamento', 'variable'])
df.head(2)

value
year departamento variable       
2015 Amazonas     generica      0
     Ancash       generica      0